# WSB Dataset Exploration

This notebook file shows an exemplary dataset analysis created for the paper submission.
The code can easily be modified to investigate further properties of the dataset, if required.


# 1. Load datasets and create filtered subset

## 1.1. WallStreetBets submissions

In [1]:
import json
import pandas as pd
import datetime
import numpy as np

In [2]:
# Load WSB submission dataset
with open('datasets/submissions_WSB_20220703.json') as f:
    submission_data = json.load(f)

In [3]:
# Created filtered subset of submissions by removing deleted and reactive posts:
def check_selftext(txt):
    return (txt in ["[deleted]", "[removed]", ""])

def check_flair(flair):
    return (flair in ["Meme", "Gain", "Loss", "Shitpost", "Satire", "Donation", "Question"])

submission_data_filtered = []
for s in submission_data:
    # Only consider posts that have a selftext
    if 'selftext' in s:
        # Only consider posts that are not deleted or removed
        if check_selftext(s['selftext']):
            pass
        else:
            # Only consider posts that have a flair with predictive nature
            if 'link_flair_text' in s:
                if check_flair(s['link_flair_text']):
                    pass
                else:
                    submission_data_filtered.append(s)
            
    

## 1.2. Combined WSB signal & stock price dataset

In [4]:
with open('datasets/stock_dfs_2022-07-03.json', 'r') as f:
    stock_dfs = json.load(f)
    
ticker_list = []

for i, s in enumerate(stock_dfs):
    stock_dfs[i] = pd.DataFrame.from_dict(s)
    stock_dfs[i].index = pd.to_datetime(stock_dfs[i].index, format='%Y-%m-%d')
    
    # Save list of tickers for further use
    ticker_list.append(stock_dfs[i].columns[0])
    
    # Rename ticker_specific columns:
    stock_dfs[i] = stock_dfs[i].rename(columns={stock_dfs[i].columns[0]: 'count', 
                                                stock_dfs[i].columns[8]: 'count_window'})
    
    

In [5]:
with open('datasets/stock_dfs_filtered_2022-07-03.json', 'r') as f:
    stock_dfs_filtered = json.load(f)
    
ticker_list_filtered = []

for i, s in enumerate(stock_dfs_filtered):
    stock_dfs_filtered[i] = pd.DataFrame.from_dict(s)
    stock_dfs_filtered[i].index = pd.to_datetime(stock_dfs_filtered[i].index, format='%Y-%m-%d')
    
    # Save list of tickers for further use
    ticker_list_filtered.append(stock_dfs_filtered[i].columns[0])
    
    # Rename ticker_specific columns:
    stock_dfs_filtered[i] = stock_dfs_filtered[i].rename(columns={stock_dfs_filtered[i].columns[0]: 'count', 
                                                                  stock_dfs_filtered[i].columns[8]: 'count_window'})
    
    

## 1.3. Preprocessed / feature engineered dataset

In [6]:
# Load dataset
submission_df = pd.read_pickle('datasets/submission_df_2022-07-03.pkl')

In [7]:
# Show columns of submission_df
print(submission_df.columns.to_list())

['all_awardings', 'allow_live_comments', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'post_hint', 'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 's

# 2. General statistics and information

## 2.1. WallStreetBets submissions

In [8]:
# Features included in the dataset
print(f'Number of features provided by the Pushshift API: {len(submission_data_filtered[0].keys())} \n')
print('List of all features:')
print(submission_data_filtered[0].keys())

Number of features provided by the Pushshift API: 67 

List of all features:
dict_keys(['all_awardings', 'allow_live_comments', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_created_from_ads_ui', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'post_hint', 'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_re

In [9]:
# Number of posts in dataset
print(f'Number of raw posts in dataset: {len(submission_data)}')

Number of raw posts in dataset: 1670273


In [10]:
# Time range of posts
t_min = submission_data[-1]['created_utc']
t_max = submission_data[0]['created_utc']

print('Earliest post from (UTC time):')
print(datetime.datetime.fromtimestamp(t_min).strftime('%Y-%m-%d %H:%M:%S'))
print('\nLatest post from (UTC time):')
print(datetime.datetime.fromtimestamp(t_max).strftime('%Y-%m-%d %H:%M:%S'))

Earliest post from (UTC time):
2017-12-31 07:21:44

Latest post from (UTC time):
2022-07-03 06:27:29


## 2.2. Combined WSB signal & stock price dataset

In [11]:
# Which columns does the stock market dataset include?
print(stock_dfs[0].columns)

Index(['count', '$count', 'BUY', 'HOLD', 'SELL', 'BUY_ngrams', 'score',
       'posts', 'count_window', 'BUY_signal', 'SELL_signal', 'activity',
       'Open', 'High', 'Low', 'Close', 'Volatility', 'Volume', 'Dividends',
       'Stock Splits', 'close_diff', 'any_buy', 'Morgan Stanley',
       'Credit Suisse', 'Wells Fargo', 'Citigroup', 'Barclays',
       'Deutsche Bank', 'UBS', 'Raymond James', 'JP Morgan',
       'B of A Securities', 'BMO Capital', 'Keybanc', 'RBC Capital',
       'Goldman Sachs', 'Mizuho', 'Stifel', 'Piper Sandler', 'Baird',
       'Jefferies', 'Oppenheimer', 'weekday', 'prev_1w', 'prev_3d', 'prev_1d',
       'change_1d', 'change_3d', 'change_1w', 'change_1m', 'change_3m', 'MA07',
       'MA30', 'MA90', 'BUY_MA30', 'any_buy_MA30'],
      dtype='object')


In [12]:
# Number of signals extracted from all WSB submissions:
signal_counts = []

for df in stock_dfs:
    signal_counts.append(df[['count', '$count', 'BUY', 'HOLD', 'SELL', 
                             'BUY_ngrams', 'posts', 'BUY_signal', 'SELL_signal']].astype(bool).sum(axis=0))
    
signal_counts_df = pd.DataFrame(signal_counts)
print(signal_counts_df)

     count  $count  BUY  HOLD  SELL  BUY_ngrams  posts  BUY_signal  \
0      105      33   43     3    17           3    105          42   
1       11       1    5     0     2           0     11           5   
2       88      27   29     1    11           2     88          29   
3      110      34   43     0    10           3    110          43   
4       20       3    9     0     4           0     20           9   
..     ...     ...  ...   ...   ...         ...    ...         ...   
498     96      30   31     1    11           4     96          31   
499     15       3    5     0     3           1     15           5   
500     11       2    4     0     2           0     11           4   
501     12       1    5     1     2           0     12           5   
502     11       1    6     0     1           0     11           6   

     SELL_signal  
0             16  
1              2  
2             11  
3             10  
4              4  
..           ...  
498           10  
499    

In [13]:
# Distribution of the different signal types:
print(signal_counts_df.sum(axis=0))

count          46178
$count         13574
BUY            18316
HOLD            1422
SELL            9152
BUY_ngrams      1910
posts          46178
BUY_signal     16649
SELL_signal     6866
dtype: int64


In [14]:
# Number of signals extracted from filtered WSB submissions:
signal_counts_filtered = []

for df in stock_dfs_filtered:
    signal_counts_filtered.append(df[['count', '$count', 'BUY', 'HOLD', 'SELL', 
                                  'BUY_ngrams', 'posts', 'BUY_signal', 'SELL_signal']].astype(bool).sum(axis=0))
    
signal_counts_filtered_df = pd.DataFrame(signal_counts_filtered)
signal_counts_filtered_df.sum(axis=0)

count          28007
$count          5785
BUY            12315
HOLD             841
SELL            5928
BUY_ngrams       895
posts          28007
BUY_signal     11522
SELL_signal     4902
dtype: int64

In [15]:
# Most frequently recommended stock tickers:
signal_counts_filtered_df['ticker'] = pd.Series(ticker_list)
signal_counts_filtered_df.sort_values('$count', ascending=False).head(10) 

,count,$count,BUY,HOLD,SELL,BUY_ngrams,posts,BUY_signal,SELL_signal,ticker
443,1001,495,605,80,423,147,1001,457,176,TSLA
45,776,222,420,25,217,42,776,359,140,AAPL
28,850,205,455,36,218,77,850,386,121,AMD
26,686,178,333,20,173,28,686,287,117,AMZN
316,613,155,332,23,161,28,613,290,103,MSFT
204,142,142,38,4,28,0,142,37,27,F
152,380,134,176,9,91,11,380,160,77,DIS
349,491,109,230,18,94,26,491,210,69,NVDA
315,337,99,170,6,48,17,337,162,36,MU
71,322,85,137,9,93,7,322,121,80,BA


In [16]:
# Top 50 stock tickers:
print(signal_counts_filtered_df.sort_values('$count', ascending=False).head(50)['ticker'].to_list())

['TSLA', 'AAPL', 'AMD', 'AMZN', 'MSFT', 'F', 'DIS', 'NVDA', 'MU', 'BA', 'MRNA', 'TWTR', 'NFLX', 'PFE', 'ATVI', 'INTC', 'T', 'WMT', 'GOOG', 'SBUX', 'GE', 'PYPL', 'TGT', 'GM', 'CRM', 'AAL', 'CCL', 'COST', 'BAC', 'XOM', 'V', 'LMT', 'JPM', 'GOOGL', 'MO', 'HD', 'GILD', 'MTCH', 'NKE', 'DAL', 'WBA', 'JNJ', 'LUV', 'KO', 'CSCO', 'BBY', 'NCLH', 'MGM', 'MCD', 'ETSY']


# 3. Post categories (flairs)

In [17]:
# Show distribution of flairs in raw posts
from collections import Counter

flairs = []
for s in submission_data:
    if 'link_flair_text' in s:
        flairs.append(s['link_flair_text'])
        
flair_counter = Counter(flairs)
flair_counter.most_common(20)

[('Discussion', 518103),
 ('Meme', 250251),
 ('YOLO', 232052),
 ('News', 131925),
 ('Gain', 128711),
 ('DD', 80582),
 ('Loss', 65877),
 ('Shitpost', 62817),
 ('Technical Analysis', 18883),
 ('Chart', 17380),
 ('Options', 15406),
 ('Stocks', 14081),
 ('Fundamentals', 6423),
 ('Satire', 4926),
 ('Storytime', 4822),
 ('Technicals', 4406),
 ('Futures', 1792),
 ('Donation', 1606),
 ('Daily Discussion', 1497),
 ('Question', 357)]

In [18]:
# Show distribution of flairs in raw posts
from collections import Counter

flairs_filtered = []
for s_f in submission_data_filtered:
    if 'link_flair_text' in s_f:
        flairs_filtered.append(s_f['link_flair_text'])
        
flair_counter_filtered = Counter(flairs_filtered)
flair_counter_filtered.most_common(20)

[('Discussion', 120758),
 ('DD', 29513),
 ('YOLO', 26651),
 ('News', 12399),
 ('Options', 5332),
 ('Technical Analysis', 4571),
 ('Stocks', 4220),
 ('Fundamentals', 2235),
 ('Storytime', 1846),
 ('Daily Discussion', 1487),
 ('Technicals', 1394),
 ('Chart', 642),
 ('Futures', 496),
 ('Weekend Discussion', 112),
 ('Earnings Thread', 66),
 ('Forex', 65),
 ('Mods', 56),
 ('News | PLTR', 38),
 ('Daily Thread', 31),
 ('WSBbooks', 24)]

# 4. Baseline prediction performances

## 4.1. WallStreetBets

In [19]:
# Extract price changes from submission_df:
wsb_performance_df = submission_df[['date', 'change_1d', 'change_3d', 
                                    'change_1w', 'change_1m', 'change_3m']].dropna().set_index('date') 

wsb_performance_df.index = pd.to_datetime(wsb_performance_df.index)

In [20]:
# Mean price change for Q3 2021:
t_min_q3 = '2021-07-01'
t_max_q3 = '2021-09-30'

wsb_performance_df.loc[t_min_q3 : t_max_q3].mean()

change_1d    0.029012
change_3d    0.040775
change_1w    0.103033
change_1m    1.654077
change_3m    5.543383
dtype: float64

In [21]:
# Mean price change for Q1 2021:
t_min_q1 = '2022-01-01'
t_max_q1 = '2022-03-31'

wsb_performance_df.loc[t_min_q1 : t_max_q1].mean()

change_1d     0.024330
change_3d    -0.412591
change_1w    -0.765980
change_1m    -3.177427
change_3m   -12.714791
dtype: float64

## 4.2. S&P 500 index

In [22]:
# It is important to have an up-to-date version of yfinance, as the API is frequently changing
# !pip install --update yfinance
import yfinance as yf

In [23]:
# Extract S&P 500 stock market data from Yahoo! Finance API
sp500_data = yf.Ticker('^GSPC')

In [24]:
# Create DataFrame with relevant time frames
# Full time frame:
t_min = '2018-01-01'
t_max = '2022-07-03'

idx = pd.date_range(t_min, t_max)

# Extract S&P 500 price history
sp500_prices = sp500_data.history(start=t_min, end=t_max).fillna(method='ffill') #.reindex(idx)
sp500_prices['SP500_change_1d'] = (sp500_prices['Close'].shift(periods=-1)*100/sp500_prices['Close'])-100
sp500_prices['SP500_change_3d'] = (sp500_prices['Close'].shift(periods=-3)*100/sp500_prices['Close'])-100
sp500_prices['SP500_change_1w'] = (sp500_prices['Close'].shift(periods=-7)*100/sp500_prices['Close'])-100
sp500_prices['SP500_change_1m'] = (sp500_prices['Close'].shift(periods=-30)*100/sp500_prices['Close'])-100
sp500_prices['SP500_change_3m'] = (sp500_prices['Close'].shift(periods=-90)*100/sp500_prices['Close'])-100
sp500_change = sp500_prices[['SP500_change_1d', 'SP500_change_3d', 'SP500_change_1w', 
                             'SP500_change_1m', 'SP500_change_3m']]

sp500_change['date'] = sp500_change.index.strftime('%Y-%m-%d')
sp500_change[:10]

# sp500_prices[f'SP500_change_{target}'] = (sp500_prices['Close'].shift(periods=-target_in_days)*100/sp500_prices['Close'])-100
# sp500_change = sp500_prices[[f'SP500_change_{target}']]


<ipython-input-24-227bce1ba179>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_change['date'] = sp500_change.index.strftime('%Y-%m-%d')


,SP500_change_1d,SP500_change_3d,SP500_change_1w,SP500_change_1m,SP500_change_3m,date
Date,,,,,,
2018-01-02 00:00:00-05:00,0.639882,1.756053,2.661538,0.104600,1.183685,2018-01-02
2018-01-03 00:00:00-05:00,0.402864,1.277152,2.697321,0.668614,0.629172,2018-01-03
2018-01-04 00:00:00-05:00,0.703377,1.002208,1.924748,0.302130,-0.460356,2018-01-04
2018-01-05 00:00:00-05:00,0.166234,0.185191,2.165764,-0.980256,-0.754240,2018-01-05
2018-01-08 00:00:00-05:00,0.130293,0.722423,1.831346,-1.687947,-1.003748,2018-01-08
2018-01-09 00:00:00-05:00,-0.111223,1.270311,2.144812,-1.720287,-1.392804,2018-01-09
2018-01-10 00:00:00-05:00,0.703365,1.025749,3.083439,-0.033837,-0.553810,2018-01-10
2018-01-11 00:00:00-05:00,0.674960,1.264652,2.586026,0.435042,-1.558055,2018-01-11
2018-01-12 00:00:00-05:00,-0.352449,0.423152,1.841193,-1.505971,-1.900409,2018-01-12


In [25]:
# Q3 2021:
sp500_change.loc[t_min_q3 : t_max_q3][['SP500_change_1d', 'SP500_change_3d', 'SP500_change_1w', 
                             'SP500_change_1m', 'SP500_change_3m']].mean()

SP500_change_1d    0.015889
SP500_change_3d    0.003129
SP500_change_1w    0.034506
SP500_change_1m    1.496383
SP500_change_3m    4.680427
dtype: float64

In [26]:
# Q1 2022:
sp500_change.loc[t_min_q1 : t_max_q1][['SP500_change_1d', 'SP500_change_3d', 'SP500_change_1w', 
                             'SP500_change_1m', 'SP500_change_3m']].mean()

SP500_change_1d    -0.077565
SP500_change_3d    -0.190417
SP500_change_1w    -0.461547
SP500_change_1m    -2.721792
SP500_change_3m   -13.021520
dtype: float64

## 4.3. Investment Banks

In [27]:
# What is the baseline performance of the investment bank recommendations?
# For Q3 2021:
top_banks = ['Morgan Stanley',
       'Credit Suisse', 'Wells Fargo', 'Citigroup', 'Barclays',
       'Deutsche Bank', 'UBS', 'Raymond James', 'JP Morgan',
       'B of A Securities', 'BMO Capital', 'Keybanc', 'RBC Capital',
       'Goldman Sachs', 'Mizuho', 'Stifel', 'Piper Sandler', 'Baird',
       'Jefferies', 'Oppenheimer']
targets = ['change_1d', 'change_3d', 'change_1w', 'change_1m', 'change_3m']

top_results_q3 = pd.DataFrame(columns=targets)
top_results_q3_count = pd.DataFrame(columns=targets)

for df in stock_dfs_filtered:
    df = df['2021-07-01':'2021-09-30']   # Filter for Q3 2021
    # top_results_q3 = top_results_q3.append(df[targets].loc[df[top_banks].any(1)].mean() + 100, ignore_index=True)
    top_results_q3 = top_results_q3.append(df[targets].loc[df[top_banks].any(1)] + 100, ignore_index=True)
    top_results_q3_count = top_results_q3_count.append(df[top_banks].loc[df[top_banks].any(1)], ignore_index=True)

top_results_q3.replace([np.inf, -np.inf], np.nan, inplace=True)
print(top_results_q3)
print(top_results_q3.mean())
print(top_results_q3_count.sum().sum())

      change_1d   change_3d   change_1w   change_1m   change_3m
0     99.865330   98.564209  100.363817  102.619654  100.025860
1    100.000000   98.940405  100.149017  104.569539  103.917907
2    100.692939   99.714173   98.796019   97.245565  100.528535
3     98.557550  101.017310  100.488924  103.926522  112.221055
4    100.000000   99.274685   98.093117   95.107038  108.959052
..          ...         ...         ...         ...         ...
728  100.678980  101.689561  102.810675  108.587932  102.100341
729  101.308771  100.519458   98.293194   98.994804   98.434261
730  105.022420  105.022420  102.511213  108.071749  121.058292
731  100.000000   98.930059   99.801856  103.868635  107.999841
732   99.519325  100.060074  102.533543  104.811736  108.646515

[733 rows x 5 columns]
change_1d    100.142971
change_3d    100.237451
change_1w    100.127829
change_1m    101.758522
change_3m    105.316333
dtype: float64
768.0


In [28]:
# For Q1 2022:
top_results_q1 = pd.DataFrame(columns=targets)
top_results_q1_count = pd.DataFrame(columns=targets)

for df in stock_dfs_filtered:
    df = df['2022-01-01':'2022-03-31']   # Filter 
    # top_results_q1 = top_results_q1.append(df[targets].loc[df[top_banks].any(1)].mean() + 100, ignore_index=True)
    top_results_q1 = top_results_q1.append(df[targets].loc[df[top_banks].any(1)] + 100, ignore_index=True)
    top_results_q1_count = top_results_q1_count.append(df[top_banks].loc[df[top_banks].any(1)], ignore_index=True)

top_results_q1.replace([np.inf, -np.inf], np.nan, inplace=True)
print(top_results_q1)
print(top_results_q1.mean())
print(top_results_q1_count.sum().sum())

      change_1d   change_3d   change_1w   change_1m   change_3m
0    100.000000   99.778691   94.874628   96.022063   91.328669
1    104.076718  104.076718  108.028894  101.635671   97.297590
2     99.528961   99.271375  101.045108  104.592262  121.532264
3     98.263188   99.992650   99.506936  104.511351  123.130717
4    100.703683  102.882876  103.578990  111.047208  118.271250
..          ...         ...         ...         ...         ...
960   99.537345  102.569399  103.833443  110.583282   84.806676
961  101.668108  101.668108  101.624594  103.234691   92.355666
962  101.083023  102.512031  103.865824  100.737061   91.892294
963  102.694855   98.424545   97.733555   95.163074   74.018796
964  100.000000   95.167457   98.195442   98.333084   87.322219

[965 rows x 5 columns]
change_1d    100.122534
change_3d    100.208456
change_1w    100.325336
change_1m     99.736536
change_3m     95.476649
dtype: float64
991.0
